# Nonnegative Matrix Factorization (NMF)
# for Time Series Forecasting and Clustering

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import sys  
import os
import random

# 0) Data Preprocessing

In [ ]:
data = pd.io.parsers.read_csv('./data/LD2011_2014.txt', sep=";", index_col=0, header=0, low_memory=False, decimal=',')
df = data
df = df.iloc[2*96:, :]
df = df.iloc[:-1, :]
df = df.iloc[:-3*96, :]

df.index = pd.to_datetime(df.index, format='%Y-%m-%d')
df = df.groupby(pd.Grouper(freq='W-MON')).sum()
df = df.iloc[:-1, :]

print(df.transpose())

periods_to_forecast = 4

X_original = df.transpose().values
n = np.shape(X_original)[0]
d = np.shape(X_original)[1]

d0 = d-periods_to_forecast
X_real = X_original.copy()

X_train = X_original[:, :d0]
X_test = X_original[:, d0:d0+periods_to_forecast]

# 1) Latent Clustered Forecast (LCF)

In [ ]:
sys.path.insert(0, './latent clustered forecast')
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import make_scorer, recall_score, precision_score, mean_absolute_error
import multiprocessing as mp
import include_lcf as nmf

sys.setrecursionlimit(10000)

In [ ]:
current_directory = os.getcwd()
matrix_dir = os.path.join(current_directory, r'matrix')
archetypes_dir = os.path.join(current_directory, r'archetypes')
trajectories_dir = os.path.join(current_directory, r'trajectories')
if not os.path.exists(matrix_dir):
    os.makedirs(matrix_dir)
if not os.path.exists(archetypes_dir):
    os.makedirs(archetypes_dir)
if not os.path.exists(trajectories_dir):
    os.makedirs(trajectories_dir)

# %%

max_depth_list = [4]
time_windows_list = [4]
list_p = [4]

best_error = 1000000

f = open("output_file_calibration.out", 'w')

for p in list_p:
    
    s = np.random.uniform(size=(n, p - 1))
    s = np.sort(s, axis=1)
    k = np.ones(n) - s[:, -1]
    j = np.diag(np.ones(p - 2), k=1)
    s = s - np.dot(s, j)
    W_init = np.c_[s, k]
    check = np.sum(W_init, axis=1)
    H_init = np.random.rand(p, np.shape(X_train)[1])

    start_time = time.time()
  
    M_HALS, H_HALS, W_HALS, objective_HALS = nmf.mnmf(p, X_train, H_init, W_init, max_iter=5000)
    
    print("elapsed time: %5.2f seconds" % (time.time() - start_time))
    X_HALS = np.maximum(0, np.dot(W_HALS, H_HALS))
    print('RRMSE =', nmf.rrmse(X_train, X_HALS))
            
    save_fileobj = ('matrix/objective_hals_rank_' + str(p) + '_entire.csv')
    save_fileM = ('matrix/matrix_hals-m_rank_' + str(p) + '_entire.csv')
    save_fileH = ('matrix/matrix_hals-h_rank_' + str(p) + '_entire.csv')
    save_fileW = ('matrix/matrix_hals-w_rank_' + str(p) + '_entire.csv')
            
    np.savetxt(save_fileobj, objective_HALS, delimiter=',')
    np.savetxt(save_fileM, M_HALS, delimiter=',')
    np.savetxt(save_fileH, H_HALS, delimiter=',')
    np.savetxt(save_fileW, W_HALS, delimiter=',')
    
    matrix = pd.read_csv('matrix/matrix_hals-w_rank_' + str(p) + '_entire.csv', delimiter=',', header=None)
    W_HALS = matrix.values

    matrix = pd.read_csv('matrix/matrix_hals-h_rank_' + str(p) + '_entire.csv', delimiter=',', header=None)
    H_HALS = matrix.values

    matrix = pd.read_csv('matrix/matrix_hals-m_rank_' + str(p) + '_entire.csv', delimiter=',', header=None)
    M_HALS = matrix.values

    model = AgglomerativeClustering(compute_full_tree=True, linkage="complete", affinity='l1')
    model.fit(W_HALS)
    n_samples = n
    minimal_size = p

    if 'rtp1' in locals() or 'rtp1' in globals():
        rtp1.clear()
        
    if 'rtp2' in locals() or 'rtp2' in globals():
        rtp2.clear()
    
    rtp = []
    rtp2 = nmf.tree_explorer(model, np.size(model.children_), n, minimal_size)
    rtp1 = rtp2.copy()
    
    for i in range(len(rtp2)):
        if (rtp2[i])[0] >= n_samples:
            rtp1[i].pop(0)     
    
    number_of_clusters = len(rtp1)
    labels = np.zeros(n)
    
    index_label = 0
    for i in range(len(rtp1)):
        for j in rtp1[i]:
            labels[j] = index_label
        index_label += 1

    W_cluster_all = np.zeros((n, p, len(rtp1)))
    
    for i in range(len(rtp1)):
        for j in rtp1[i]:
            for k in range(p):
                W_cluster_all[j, k, i] = W_HALS[j, k]
    
    W_cluster_final = [None] * len(rtp1)
    
    for i in range(len(rtp1)):
        a = W_cluster_all[:, :, i]
        W_cluster_final[i] = a[~(a==0).all(1)]
    
    X_cluster_all = np.zeros((n, d0, len(rtp1)))
    
    for i in range(len(rtp1)):
        for j in rtp1[i]:
            for k in range(d0):
                X_cluster_all[j, k, i] = X_train[j, k]
                
    X_c = [None] * len(rtp1)
    
    for i in range(len(rtp1)):
        a = X_cluster_all[:, :, i]
        X_c[i] = a[~(a==0).all(1)]
   
    cluster_with_full_rank = []
    cluster_small = []
    
    for i in range(len(rtp1)):
        if len(rtp1[i]) == 1:    
            cluster_small.append(i)
        cluster_with_full_rank.append(i)

    X_c_new = [None] * len(rtp1)
    H_test = [None] * len(cluster_with_full_rank)
    W_test = [None] * len(cluster_with_full_rank)
    cluster_index = 0
      
    print('total number of clusters:', len(cluster_with_full_rank))
    print('number of processes:', mp.cpu_count())

    pool = mp.Pool(mp.cpu_count())        
    X_c_new = pool.starmap(nmf.archetypes, [(X_c[i], p, i) for i in cluster_with_full_rank])
    pool.close() 

    for cluster_index in cluster_with_full_rank:
        matrix = pd.read_csv('matrix/H_test_cluster_' + str(p) + '_' + str(cluster_index) + '_entire.csv', delimiter=',', header=None)
        H_test[cluster_index] = matrix.values
        
        matrix = pd.read_csv('matrix/W_test_cluster_' + str(p) + '_' + str(cluster_index) + '_entire.csv', delimiter=',', header=None)
        W_test[cluster_index] = matrix.values
        
        X_c_new[cluster_index] = np.dot(W_test[cluster_index], H_test[cluster_index])

    model = 'random_forest'
    cluster_index = len(cluster_with_full_rank)

    time_window_index = 0
    H_pred = [None] * cluster_index
    X_c_final = [None] * cluster_index

    for time_window in time_windows_list: 
        for max_depth in max_depth_list:
            
            max_depth_list1 = []
            max_depth_list1.append(max_depth)
            
            rdf = RandomForestRegressor(criterion='mse')
            clf = GridSearchCV(estimator=rdf, param_grid=dict(max_depth=max_depth_list1), n_jobs=-1, scoring='neg_mean_squared_error')
        
            for c in range(cluster_index):
                H_pred[c] = np.zeros((p, d-d0))
                for i in range(p):    

                    time_window = int(np.floor(time_window))
                    #print('processing time window:', time_window)
                    #print('processing max_depth:', max_depth)
                    #print('processing archetype', i, 'of cluster', c, 'for rank', p)

                    H_random_forest = (H_test[c])[i, :d0]
                    X, y = nmf.split_sequence(H_random_forest, time_window, d-d0) 
                    clf.fit(X, y) 
		
                    H_test1 = (H_test[c])[i, d0-time_window:d0]
                    H_test1 = H_test1.reshape((1, time_window))
                    (H_pred[c])[i, :] = clf.predict(H_test1)
                    #print('forecasts:', (H_pred[c])[i, :])
                    
                    save_fileH_pred = ('archetypes/matrix_hals-h_pred_' + str(time_window) + '_' + str(max_depth) + '_rank_' + str(p) + '_' + str(c) + '_' + str(i) + '_entire.csv')
                    np.savetxt(save_fileH_pred, (H_pred[c])[i, :], delimiter=',')
               
                    matrix = pd.read_csv('archetypes/matrix_hals-h_pred_' + str(time_window) + '_' + str(max_depth) + '_rank_' + str(p) + '_' + str(c) + '_' + str(i) + '_entire.csv', delimiter=',', header=None)		    
                    (H_pred[c])[i, :] = matrix.values[:, 0]
                
                X_c_final[c] = np.maximum(0, np.dot(W_test[c], H_pred[c]))
                
            X_HALS_final = np.zeros((n, d-d0))            
                    
            for i in range(len(rtp1)):
                index = 0
                for j in rtp1[i]:
                    for k in range(d-d0): 
                        X_HALS_final[j, k] = (X_c_final[i])[index, k]
                    index += 1
            
            print('rank p:', p) 
            print('max_depth:', max_depth)
            print('time_window:', time_window)
	    
            print('rrmse:', nmf.rrmse(X_test, X_HALS_final))
            print('mpe:', nmf.mpe(X_test, X_HALS_final))
            print('l2:', nmf.l2_error(X_test, X_HALS_final))
            print('l1:', nmf.l1_error(X_test, X_HALS_final))
            
            f.write('rank p: ' + str(p) + '\n')
            f.write('max_depth: ' + str(max_depth) + '\n')
            f.write('time_window: ' + str(time_window) + '\n')
            
            f.write('rrmse: ' + str(nmf.rrmse(X_test, X_HALS_final)) + '\n')
            f.write('mpe: ' + str(nmf.mpe(X_test, X_HALS_final)) + '\n')
            f.write('l2: ' + str(nmf.l2_error(X_test, X_HALS_final)) + '\n')
            f.write('l1: ' + str(nmf.l1_error(X_test, X_HALS_final)) + '\n\n')
            
            if nmf.rrmse(X_test, X_HALS_final) < best_error:
                p_best = p
                max_depth_best = max_depth
                time_window_best = time_window
                X_forecast_best = X_HALS_final.copy()
                best_error = nmf.rrmse(X_test, X_HALS_final)
                
save_file_forecast_best = ('matrix_forecast_best_entire.csv')
np.savetxt(save_file_forecast_best, X_forecast_best, delimiter=',')
                
print('best rank p:', p_best) 
print('best max_depth:', max_depth_best)
print('best time_window:', time_window_best)
print('best RRMSE:', best_error)

f.write('best rank p: ' + str(p_best) + '\n')
f.write('best max_depth: ' + str(max_depth_best) + '\n')
f.write('best time_window: ' + str(time_window_best) + '\n\n')
                
f.close()

# %%

trajectory_list = random.sample(range(n), 5)

for i in trajectory_list:
    nmf.trajectory_plot(i, X_test, X_forecast_best, p_best)
    print(nmf.mpe(X_test[i, :], X_forecast_best[i, :]))


# 2) Sliding Mask Method (SMM)

In [ ]:
sys.path.insert(0, './sliding mask method')

### 2a) mask Archetypal Matrix Factorization (mAMF) without overlapping

In [ ]:
import include_amf as amf

current_directory = os.getcwd()
mask_dir = os.path.join(current_directory, r'mask_test')
if not os.path.exists(mask_dir):
    os.makedirs(mask_dir)
trajectories_dir = os.path.join(current_directory, r'trajectories')
if not os.path.exists(trajectories_dir):
    os.makedirs(trajectories_dir)
    
# %%

best_error = 1000000
f = open("mask_test/output_file.out", 'w')

list_w = [4]
list_rank = [5]

for w in list_w:
    print("w:", w)
    f.write('w: ' + str(w) +'\n\n')
    
    periodicity = 4
    X_real = X_original.copy()
    for i in range(n):
        X_real[i, d0:d] = X_original[i, d0-periods_to_forecast:d0]
    X_real = X_real.flatten(order='C')
    
    # %%
    
    b=d/(periodicity*w)
    row = int(b*n)
    
    X_original_block = np.split(X_real, np.shape(X_real)[0]/periodicity)    
    X_original_block_2 = np.zeros((row, w*periodicity))
    tau_2 = np.zeros(row)
    
    jindex = 0
    a = 0
    for index in range(len(X_original_block)):
        c = X_original_block[index]
        X_original_block_2[jindex, a:a+periodicity] = c
        a += periodicity
        if a == w*periodicity:
            jindex += 1
            a = 0
    
    index2 = b-1
    for jndex1 in range(int(row/b)):
        index2 = int(index2)
        tau_2[index2] = 1
        index2 += b
    
    tau = np.argmax(X_original_block_2 != 0, axis=1)
    
    tau_1a = tau_2[(~np.all(X_original_block_2 == 0, axis=1)) | (tau_2==1)]
    X_original_block_1 = X_original_block_2[(~np.all(X_original_block_2 == 0, axis=1)) | (tau_2==1)] 
    
    # %%
        
    m = np.shape(X_original_block_1)[0]
    
    for p in list_rank:
        np.random.seed(0)
    
        s = np.random.uniform(size=(m, p - 1))
        s = np.sort(s, axis=1)
        k = np.ones(m) - s[:, -1]
        j = np.diag(np.ones(p - 2), k=1)
        s = s - np.dot(s, j)
        W_init = np.c_[s, k]
        check = np.sum(W_init, axis=1)
        
        H_init = np.random.rand(p, w*periodicity)*100
    
        start_time = time.time()
    
        print("rank:", p)
    
        M_HALS, H_HALS, W_HALS, objective_HALS = amf.mnmf(p, X_original_block_1, H_init, W_init, tau_1a, periods_to_forecast, max_iter=5000)
        print('Error: %5.4f' % np.linalg.norm(M_HALS - np.dot(W_HALS, H_HALS)))
        print("elapsed time: %5.2f seconds" % (time.time() - start_time))
    
        save_fileobj = ('mask_test/objective_hals_rank_' + str(w) + '_' + str(p) +'.csv')
        save_fileM = ('mask_test/matrix_hals-m_rank_' + str(w) + '_' + str(p) +'.csv')
        save_fileH = ('mask_test/matrix_hals-h_rank_' + str(w) + '_' + str(p) +'.csv')
        save_fileW = ('mask_test/matrix_hals-w_rank_' + str(w) + '_' + str(p) +'.csv')
    
        np.savetxt(save_fileobj, objective_HALS, delimiter=',')
        np.savetxt(save_fileM, M_HALS, delimiter=',')
        np.savetxt(save_fileH, H_HALS, delimiter=',')
        np.savetxt(save_fileW, W_HALS, delimiter=',')
    
        matrix = pd.read_csv('mask_test/matrix_hals-w_rank_' + str(w) + '_' + str(p) +'.csv', delimiter=',', header=None)
        W_HALS = matrix.values
    
        matrix = pd.read_csv('mask_test/matrix_hals-h_rank_' + str(w) + '_' + str(p) +'.csv', delimiter=',', header=None)
        H_HALS = matrix.values
    
        matrix = pd.read_csv('mask_test/matrix_hals-m_rank_' + str(w) + '_' + str(p) +'.csv', delimiter=',', header=None)
        M_HALS = matrix.values
        
        M_new_block = np.zeros((np.shape(X_original_block_2)[0], w*periodicity))
        M_new_block[(~np.all(X_original_block_2 == 0, axis=1)) | (tau_2==1)] = M_HALS[:]
        M_new = np.zeros((n, d))
    
        jindex = 0
        a = 0
        for index in range(np.shape(X_original_block_2)[0]):
            c = M_new_block[index, :]
            M_new[jindex, a:a+periodicity*w] = c
            a += periodicity*w
            if a == d:
                jindex += 1
                a = 0
                
        M_new = M_new[:, d0:d] 
        
        prevision_error_HALS_rrmse = amf.rrmse(X_test, M_new)
        prevision_error_HALS_l1 = amf.l1_error(X_test, M_new)
        prevision_error_HALS_mpe = amf.mpe(X_test, M_new)
        prevision_error_HALS_l2 = amf.l2_error(X_test, M_new)
        
        print('rrmse:', prevision_error_HALS_rrmse)
        print('mpe:', prevision_error_HALS_mpe)
        print('l2:', prevision_error_HALS_l2)
        print('l1:', prevision_error_HALS_l1)  
    
        f.write('p: ' + str(p)  + '\n')
        f.write('rrmse: ' + str(prevision_error_HALS_rrmse)  + '\n')
        f.write('mpe: ' + str(prevision_error_HALS_mpe)  + '\n')
        f.write('l2: ' + str(prevision_error_HALS_l2)  + '\n')
        f.write('l1: ' + str(prevision_error_HALS_l1)  + '\n')
        f.write('\n')   
    
        if amf.rrmse(X_test, M_new) <= best_error:
            w_best = w
            p_best = p
            X_forecast_best = M_new.copy()
            best_error = amf.rrmse(X_test, M_new)
                    
save_file_forecast_best = ('matrix_forecast_best_mask_test.csv')
np.savetxt(save_file_forecast_best, X_forecast_best, delimiter=',')
                    
print('best rank w:', w_best) 
print('best rank p:', p_best) 
print('best RRMSE:', best_error)

f.write('best rank w: ' + str(w_best) + '\n')    
f.write('best rank p: ' + str(p_best) + '\n')
f.write('best RRMSE: ' + str(best_error) + '\n')
                    
f.close()
    
# %%
    
trajectory_list = random.sample(range(n), 5)

for i in trajectory_list:
    print(amf.rrmse(X_test[i, :], X_forecast_best[i, :]))
    amf.trajectory_plot(i, X_test, X_forecast_best, p_best)

### 2b) mask Archetypal Matrix Factorization (mAMF) with overlapping

In [ ]:
import include_amf_overlap as amf_overlap

current_directory = os.getcwd()
mask_dir = os.path.join(current_directory, r'mask_test')
if not os.path.exists(mask_dir):
    os.makedirs(mask_dir)
trajectories_dir = os.path.join(current_directory, r'trajectories')
if not os.path.exists(trajectories_dir):
    os.makedirs(trajectories_dir)
    
# %%

best_error = 1000000
f = open("mask_test/output_file.out", 'w')

list_w = [4]
list_rank = [5]

for w in list_w:
    print("w:", w)
    f.write('w: ' + str(w) +'\n\n')
    
    periodicity = 4
    X_real = X_original.copy()
    for i in range(n):
        X_real[i, d0:d] = X_original[i, d0-periods_to_forecast:d0]
    X_real = X_real.flatten(order='C')
    
    # %%
    
    b=d/(periodicity*w)
    row = int(b*n*2)
    size = int(periodicity)
    step = int(periodicity/2)

    X_original_block = [X_real[i : i + size] for i in range(0, len(X_real), step)]
    X_original_block_2 = np.zeros((row, w*periodicity))
    tau_2 = np.zeros(row)

    pp = int(periodicity/2)
    
    jindex = 0
    a = 0
    for index in range(len(X_original_block)-1):
        c = X_original_block[index]
        X_original_block_2[jindex, a:a+periodicity] = c
        a += periodicity
        if a == w*periodicity:
            jindex += 1
            a = 0
    
    index2 = b*2-1
    for jndex1 in range(int(row/b/2)):
        index2 = int(index2)
        tau_2[index2] = 1
        index2 += b*2

    tau = np.argmax(X_original_block_2 != 0, axis=1)    
    tau_1a = tau_2[(~np.all(X_original_block_2 == 0, axis=1)) | (tau_2==1)]
    X_original_block_1 = X_original_block_2[(~np.all(X_original_block_2 == 0, axis=1)) | (tau_2==1)]

    
    # %%
        
    m = np.shape(X_original_block_1)[0]
    
    for p in list_rank:
        np.random.seed(0)
    
        s = np.random.uniform(size=(m, p - 1))
        s = np.sort(s, axis=1)
        k = np.ones(m) - s[:, -1]
        j = np.diag(np.ones(p - 2), k=1)
        s = s - np.dot(s, j)
        W_init = np.c_[s, k]
        check = np.sum(W_init, axis=1)
        
        H_init = np.random.rand(p, w*periodicity)*100
    
        start_time = time.time()
    
        print("rank:", p)
    
        M_HALS, H_HALS, W_HALS, objective_HALS = amf_overlap.mnmf(p, X_original_block_1, H_init, W_init, tau_1a, w, periodicity, periods_to_forecast, max_iter=5000)
        print('Error: %5.4f' % np.linalg.norm(M_HALS - np.dot(W_HALS, H_HALS)))
        print("elapsed time: %5.2f seconds" % (time.time() - start_time))
    
        save_fileobj = ('mask_test/objective_hals_rank_' + str(w) + '_' + str(p) +'.csv')
        save_fileM = ('mask_test/matrix_hals-m_rank_' + str(w) + '_' + str(p) +'.csv')
        save_fileH = ('mask_test/matrix_hals-h_rank_' + str(w) + '_' + str(p) +'.csv')
        save_fileW = ('mask_test/matrix_hals-w_rank_' + str(w) + '_' + str(p) +'.csv')
    
        np.savetxt(save_fileobj, objective_HALS, delimiter=',')
        np.savetxt(save_fileM, M_HALS, delimiter=',')
        np.savetxt(save_fileH, H_HALS, delimiter=',')
        np.savetxt(save_fileW, W_HALS, delimiter=',')
    
        matrix = pd.read_csv('mask_test/matrix_hals-w_rank_' + str(w) + '_' + str(p) +'.csv', delimiter=',', header=None)
        W_HALS = matrix.values
    
        matrix = pd.read_csv('mask_test/matrix_hals-h_rank_' + str(w) + '_' + str(p) +'.csv', delimiter=',', header=None)
        H_HALS = matrix.values
    
        matrix = pd.read_csv('mask_test/matrix_hals-m_rank_' + str(w) + '_' + str(p) +'.csv', delimiter=',', header=None)
        M_HALS = matrix.values
        
        M_new_block = np.zeros((np.shape(X_original_block_2)[0], w*periodicity))
        M_new_block[(~np.all(X_original_block_2 == 0, axis=1)) | (tau_2==1)] =  M_HALS[:]
        M_new = np.zeros((n, d-d0))
        
        jindex = 0
        
        for i in range(np.shape(M_new_block)[0]):
            if tau_2[i] == 1:
                M_new[jindex, :] = M_new_block[i, (w-1)*periodicity-periods_to_forecast:(w-1)*periodicity]
                jindex += 1
        
        prevision_error_HALS_rrmse = amf.rrmse(X_test, M_new)
        prevision_error_HALS_l1 = amf.l1_error(X_test, M_new)
        prevision_error_HALS_mpe = amf.mpe(X_test, M_new)
        prevision_error_HALS_l2 = amf.l2_error(X_test, M_new)
        
        print('rrmse:', prevision_error_HALS_rrmse)
        print('mpe:', prevision_error_HALS_mpe)
        print('l2:', prevision_error_HALS_l2)
        print('l1:', prevision_error_HALS_l1)  
    
        f.write('p: ' + str(p)  + '\n')
        f.write('rrmse: ' + str(prevision_error_HALS_rrmse)  + '\n')
        f.write('mpe: ' + str(prevision_error_HALS_mpe)  + '\n')
        f.write('l2: ' + str(prevision_error_HALS_l2)  + '\n')
        f.write('l1: ' + str(prevision_error_HALS_l1)  + '\n')
        f.write('\n')   
    
        if amf.rrmse(X_test, M_new) <= best_error:
            w_best = w
            p_best = p
            X_forecast_best = M_new.copy()
            best_error = amf.rrmse(X_test, M_new)
                    
save_file_forecast_best = ('matrix_forecast_best_mask_test.csv')
np.savetxt(save_file_forecast_best, X_forecast_best, delimiter=',')
                    
print('best rank w:', w_best) 
print('best rank p:', p_best) 
print('best RRMSE:', best_error)

f.write('best rank w: ' + str(w_best) + '\n')    
f.write('best rank p: ' + str(p_best) + '\n')
f.write('best RRMSE: ' + str(best_error) + '\n')
                    
f.close()
    
# %%
    
trajectory_list = random.sample(range(n), 5)

for i in trajectory_list:
    print(amf.rrmse(X_test[i, :], X_forecast_best[i, :]))
    amf.trajectory_plot(i, X_test, X_forecast_best, p_best)

### 2c) mask Nonnegative Matrix Factorization (mNMF) without overlapping

In [ ]:
import include_nmf as nmf

current_directory = os.getcwd()
mask_dir = os.path.join(current_directory, r'mask_test')
if not os.path.exists(mask_dir):
    os.makedirs(mask_dir)
trajectories_dir = os.path.join(current_directory, r'trajectories')
if not os.path.exists(trajectories_dir):
    os.makedirs(trajectories_dir)

# %%

# %%

best_error = 1000000
f = open("mask_test/output_file.out", 'w')

list_w = [4]
list_rank = [5]

for w in list_w:
    print("w:", w)
    f.write('w: ' + str(w) +'\n\n')
    
    periodicity = 4
    X_real = X_original.copy()
    for i in range(n):
        X_real[i, d0:d] = X_original[i, d0-periods_to_forecast:d0]
    X_real = X_real.flatten(order='C')
    
    # %%
    
    b=d/(periodicity*w)
    row = int(b*n)
    
    X_original_block = np.split(X_real, np.shape(X_real)[0]/periodicity)    
    X_original_block_2 = np.zeros((row, w*periodicity))
    tau_2 = np.zeros(row)
    
    jindex = 0
    a = 0
    for index in range(len(X_original_block)):
        c = X_original_block[index]
        X_original_block_2[jindex, a:a+periodicity] = c
        a += periodicity
        if a == w*periodicity:
            jindex += 1
            a = 0
    
    index2 = b-1
    for jndex1 in range(int(row/b)):
        index2 = int(index2)
        tau_2[index2] = 1
        index2 += b
    
    tau = np.argmax(X_original_block_2 != 0, axis=1)
    
    tau_1a = tau_2[(~np.all(X_original_block_2 == 0, axis=1)) | (tau_2==1)]
    X_original_block_1 = X_original_block_2[(~np.all(X_original_block_2 == 0, axis=1)) | (tau_2==1)] 
    
    # %%
        
    m = np.shape(X_original_block_1)[0]
    
    for p in list_rank:
        np.random.seed(0)
    
        s = np.random.uniform(size=(m, p - 1))
        s = np.sort(s, axis=1)
        k = np.ones(m) - s[:, -1]
        j = np.diag(np.ones(p - 2), k=1)
        s = s - np.dot(s, j)
        W_init = np.c_[s, k]
        check = np.sum(W_init, axis=1)
        
        H_init = np.random.rand(p, w*periodicity)*100
    
        start_time = time.time()
    
        print("rank:", p)
    
        M_HALS, H_HALS, W_HALS, objective_HALS = nmf.mnmf(p, X_original_block_1, H_init, W_init, tau_1a, periods_to_forecast, max_iter=5000)
        print('Error: %5.4f' % np.linalg.norm(M_HALS - np.dot(W_HALS, H_HALS)))
        print("elapsed time: %5.2f seconds" % (time.time() - start_time))
    
        save_fileobj = ('mask_test/objective_hals_rank_' + str(w) + '_' + str(p) +'.csv')
        save_fileM = ('mask_test/matrix_hals-m_rank_' + str(w) + '_' + str(p) +'.csv')
        save_fileH = ('mask_test/matrix_hals-h_rank_' + str(w) + '_' + str(p) +'.csv')
        save_fileW = ('mask_test/matrix_hals-w_rank_' + str(w) + '_' + str(p) +'.csv')
    
        np.savetxt(save_fileobj, objective_HALS, delimiter=',')
        np.savetxt(save_fileM, M_HALS, delimiter=',')
        np.savetxt(save_fileH, H_HALS, delimiter=',')
        np.savetxt(save_fileW, W_HALS, delimiter=',')
    
        matrix = pd.read_csv('mask_test/matrix_hals-w_rank_' + str(w) + '_' + str(p) +'.csv', delimiter=',', header=None)
        W_HALS = matrix.values
    
        matrix = pd.read_csv('mask_test/matrix_hals-h_rank_' + str(w) + '_' + str(p) +'.csv', delimiter=',', header=None)
        H_HALS = matrix.values
    
        matrix = pd.read_csv('mask_test/matrix_hals-m_rank_' + str(w) + '_' + str(p) +'.csv', delimiter=',', header=None)
        M_HALS = matrix.values
        
        M_new_block = np.zeros((np.shape(X_original_block_2)[0], w*periodicity))
        M_new_block[(~np.all(X_original_block_2 == 0, axis=1)) | (tau_2==1)] = M_HALS[:]
        M_new = np.zeros((n, d))
    
        jindex = 0
        a = 0
        for index in range(np.shape(X_original_block_2)[0]):
            c = M_new_block[index, :]
            M_new[jindex, a:a+periodicity*w] = c
            a += periodicity*w
            if a == d:
                jindex += 1
                a = 0
                
        M_new = M_new[:, d0:d] 
        
        prevision_error_HALS_rrmse = amf.rrmse(X_test, M_new)
        prevision_error_HALS_l1 = amf.l1_error(X_test, M_new)
        prevision_error_HALS_mpe = amf.mpe(X_test, M_new)
        prevision_error_HALS_l2 = amf.l2_error(X_test, M_new)
        
        print('rrmse:', prevision_error_HALS_rrmse)
        print('mpe:', prevision_error_HALS_mpe)
        print('l2:', prevision_error_HALS_l2)
        print('l1:', prevision_error_HALS_l1)  
    
        f.write('p: ' + str(p)  + '\n')
        f.write('rrmse: ' + str(prevision_error_HALS_rrmse)  + '\n')
        f.write('mpe: ' + str(prevision_error_HALS_mpe)  + '\n')
        f.write('l2: ' + str(prevision_error_HALS_l2)  + '\n')
        f.write('l1: ' + str(prevision_error_HALS_l1)  + '\n')
        f.write('\n')   
    
        if amf.rrmse(X_test, M_new) <= best_error:
            w_best = w
            p_best = p
            X_forecast_best = M_new.copy()
            best_error = amf.rrmse(X_test, M_new)
                    
save_file_forecast_best = ('matrix_forecast_best_mask_test.csv')
np.savetxt(save_file_forecast_best, X_forecast_best, delimiter=',')
                    
print('best rank w:', w_best) 
print('best rank p:', p_best) 
print('best RRMSE:', best_error)

f.write('best rank w: ' + str(w_best) + '\n')    
f.write('best rank p: ' + str(p_best) + '\n')
f.write('best RRMSE: ' + str(best_error) + '\n')
                    
f.close()
    
# %%
    
trajectory_list = random.sample(range(n), 5)

for i in trajectory_list:
    print(amf.rrmse(X_test[i, :], X_forecast_best[i, :]))
    amf.trajectory_plot(i, X_test, X_forecast_best, p_best)

### 2d) mask Nonnegative Matrix Factorization (mNMF) with overlapping

In [ ]:
import include_nmf_overlap as nmf_overlap

current_directory = os.getcwd()
mask_dir = os.path.join(current_directory, r'mask_test')
if not os.path.exists(mask_dir):
    os.makedirs(mask_dir)
trajectories_dir = os.path.join(current_directory, r'trajectories')
if not os.path.exists(trajectories_dir):
    os.makedirs(trajectories_dir)
    
# %%

best_error = 1000000
f = open("mask_test/output_file.out", 'w')

list_w = [4]
list_rank = [5]

for w in list_w:
    print("w:", w)
    f.write('w: ' + str(w) +'\n\n')
    
    periodicity = 4
    X_real = X_original.copy()
    for i in range(n):
        X_real[i, d0:d] = X_original[i, d0-periods_to_forecast:d0]
    X_real = X_real.flatten(order='C')
    
    # %%
    
    b=d/(periodicity*w)
    row = int(b*n*2)
    size = int(periodicity)
    step = int(periodicity/2)

    X_original_block = [X_real[i : i + size] for i in range(0, len(X_real), step)]
    X_original_block_2 = np.zeros((row, w*periodicity))
    tau_2 = np.zeros(row)

    pp = int(periodicity/2)
    
    jindex = 0
    a = 0
    for index in range(len(X_original_block)-1):
        c = X_original_block[index]
        X_original_block_2[jindex, a:a+periodicity] = c
        a += periodicity
        if a == w*periodicity:
            jindex += 1
            a = 0
    
    index2 = b*2-1
    for jndex1 in range(int(row/b/2)):
        index2 = int(index2)
        tau_2[index2] = 1
        index2 += b*2

    tau = np.argmax(X_original_block_2 != 0, axis=1)    
    tau_1a = tau_2[(~np.all(X_original_block_2 == 0, axis=1)) | (tau_2==1)]
    X_original_block_1 = X_original_block_2[(~np.all(X_original_block_2 == 0, axis=1)) | (tau_2==1)]

    
    # %%
        
    m = np.shape(X_original_block_1)[0]
    
    for p in list_rank:
        np.random.seed(0)
    
        s = np.random.uniform(size=(m, p - 1))
        s = np.sort(s, axis=1)
        k = np.ones(m) - s[:, -1]
        j = np.diag(np.ones(p - 2), k=1)
        s = s - np.dot(s, j)
        W_init = np.c_[s, k]
        check = np.sum(W_init, axis=1)
        
        H_init = np.random.rand(p, w*periodicity)*100
    
        start_time = time.time()
    
        print("rank:", p)
    
        M_HALS, H_HALS, W_HALS, objective_HALS = nmf_overlap.mnmf(p, X_original_block_1, H_init, W_init, tau_1a, w, periodicity, periods_to_forecast, max_iter=5000)
        print('Error: %5.4f' % np.linalg.norm(M_HALS - np.dot(W_HALS, H_HALS)))
        print("elapsed time: %5.2f seconds" % (time.time() - start_time))
    
        save_fileobj = ('mask_test/objective_hals_rank_' + str(w) + '_' + str(p) +'.csv')
        save_fileM = ('mask_test/matrix_hals-m_rank_' + str(w) + '_' + str(p) +'.csv')
        save_fileH = ('mask_test/matrix_hals-h_rank_' + str(w) + '_' + str(p) +'.csv')
        save_fileW = ('mask_test/matrix_hals-w_rank_' + str(w) + '_' + str(p) +'.csv')
    
        np.savetxt(save_fileobj, objective_HALS, delimiter=',')
        np.savetxt(save_fileM, M_HALS, delimiter=',')
        np.savetxt(save_fileH, H_HALS, delimiter=',')
        np.savetxt(save_fileW, W_HALS, delimiter=',')
    
        matrix = pd.read_csv('mask_test/matrix_hals-w_rank_' + str(w) + '_' + str(p) +'.csv', delimiter=',', header=None)
        W_HALS = matrix.values
    
        matrix = pd.read_csv('mask_test/matrix_hals-h_rank_' + str(w) + '_' + str(p) +'.csv', delimiter=',', header=None)
        H_HALS = matrix.values
    
        matrix = pd.read_csv('mask_test/matrix_hals-m_rank_' + str(w) + '_' + str(p) +'.csv', delimiter=',', header=None)
        M_HALS = matrix.values
        
        M_new_block = np.zeros((np.shape(X_original_block_2)[0], w*periodicity))
        M_new_block[(~np.all(X_original_block_2 == 0, axis=1)) | (tau_2==1)] =  M_HALS[:]
        M_new = np.zeros((n, d-d0))
        
        jindex = 0
        
        for i in range(np.shape(M_new_block)[0]):
            if tau_2[i] == 1:
                M_new[jindex, :] = M_new_block[i, (w-1)*periodicity-periods_to_forecast:(w-1)*periodicity]
                jindex += 1
        
        prevision_error_HALS_rrmse = amf.rrmse(X_test, M_new)
        prevision_error_HALS_l1 = amf.l1_error(X_test, M_new)
        prevision_error_HALS_mpe = amf.mpe(X_test, M_new)
        prevision_error_HALS_l2 = amf.l2_error(X_test, M_new)
        
        print('rrmse:', prevision_error_HALS_rrmse)
        print('mpe:', prevision_error_HALS_mpe)
        print('l2:', prevision_error_HALS_l2)
        print('l1:', prevision_error_HALS_l1)  
    
        f.write('p: ' + str(p)  + '\n')
        f.write('rrmse: ' + str(prevision_error_HALS_rrmse)  + '\n')
        f.write('mpe: ' + str(prevision_error_HALS_mpe)  + '\n')
        f.write('l2: ' + str(prevision_error_HALS_l2)  + '\n')
        f.write('l1: ' + str(prevision_error_HALS_l1)  + '\n')
        f.write('\n')   
    
        if amf.rrmse(X_test, M_new) <= best_error:
            w_best = w
            p_best = p
            X_forecast_best = M_new.copy()
            best_error = amf.rrmse(X_test, M_new)
                    
save_file_forecast_best = ('matrix_forecast_best_mask_test.csv')
np.savetxt(save_file_forecast_best, X_forecast_best, delimiter=',')
                    
print('best rank w:', w_best) 
print('best rank p:', p_best) 
print('best RRMSE:', best_error)

f.write('best rank w: ' + str(w_best) + '\n')    
f.write('best rank p: ' + str(p_best) + '\n')
f.write('best RRMSE: ' + str(best_error) + '\n')
                    
f.close()
    
# %%
    
trajectory_list = random.sample(range(n), 5)

for i in trajectory_list:
    print(amf.rrmse(X_test[i, :], X_forecast_best[i, :]))
    amf.trajectory_plot(i, X_test, X_forecast_best, p_best)

# 3) Random Forest Regression (RFR)/Neural Network (NN)

In [ ]:
sys.path.insert(0, './random forest')
import tensorflow as tf
from keras.layers import SimpleRNN, LSTM, GRU, Bidirectional, Conv1D, MaxPooling1D, Dropout
from keras.models import Sequential
from keras.layers import Dense
import include_rfr as rfr

In [ ]:
tau = np.argmax(X_original != 0, axis=1)

current_directory = os.getcwd()
trajectories_dir = os.path.join(current_directory, r'trajectories')
if not os.path.exists(trajectories_dir):
    os.makedirs(trajectories_dir)
    
X_pred = np.zeros((n, d-d0))
        
for i in range(n):
    np.random.seed(0)
    tf.set_random_seed(1)
    #print('processing MT:', i+1)
    A = X_train[i, tau[i]:]
    X_pred[i, :] = rfr.ext_archetype(d, d0, A, time_window=np.minimum(16, int(np.floor(np.shape(A)[0]/2))), model_dl=LSTM, cnn=False, rfr=True)  
    #print('RRMSE =', rfr.rrmse(X_test[i, :], X_pred[i, :]))
    #print('forecasts:', X_pred[i, :])
    #print('test:', X_test[i, :])
            
print('RRMSE =', rfr.rrmse(X_test, X_pred))
print('MPE =', rfr.mpe(X_test, X_pred))

save_file_forecast_best = ('matrix_forecast_best_plain.csv')
np.savetxt(save_file_forecast_best, X_pred, delimiter=',')

# %%

trajectory_list = random.sample(range(n), 5)

for i in trajectory_list:
    rfr.trajectory_plot(i, X_test, X_pred)
    print(rfr.mpe(X_test[i, :], X_pred[i, :]))
